In [2]:
import numpy as np
from scipy.spatial import distance

Multi-Instance Multi-Label Learning with Application to Scene Classification

In [3]:
def haussdorf_distance(A, B, metric='euclidean'):
    """Compute the Haussdorf distance between two bags.
    
    Args:
        A (ndarray): A Na by F ndarray, where Na is the number of instances in the bag and F the number of features.
        B (nd.array): A Nb by F ndarray, where Nb is the number of instances in the bag and F the number of features.
        
    Return:
        The Haussdorf distance between the bags.
    """
    distances = distance.cdist(A, B)         
    max_a = np.max(np.min(distances, axis=1))
    max_b = np.max(np.min(distances, axis=0))
    return max(max_a, max_b)

In [4]:
rA = np.random.randn(100, 1024)
rB = np.random.randn(30, 1024)

In [5]:
%timeit haussdorf_distance(rA, rB)

100 loops, best of 3: 3.33 ms per loop


In [8]:
def gen_random_bags(n_bags=1000, n_instances_min=2, n_instances_max=200, n_features=1024, seed=42):
    np.random.seed(seed)
    ns_instance = np.random.randint(n_instances_min, n_instances_max, size=(n_bags, ))
    return [np.random.randn(ni, n_features) for ni in ns_instance]

In [14]:
tmp = np.zeros((100, 10))

In [15]:
np.argmin(tmp, axis=1).shape

(100,)

In [10]:
g = np.random.randint(0, 10, size=(1000))

In [13]:
np.where(g)[0]

array([  0,   5,  31,  34,  42,  43,  44,  63,  79,  82,  86, 110, 112,
       113, 128, 140, 142, 148, 153, 168, 195, 233, 234, 238, 239, 249,
       250, 260, 262, 269, 322, 324, 337, 355, 377, 389, 392, 393, 402,
       421, 441, 450, 456, 463, 470, 476, 505, 512, 515, 525, 534, 536,
       549, 576, 585, 596, 610, 622, 625, 643, 652, 654, 670, 686, 699,
       708, 709, 735, 754, 755, 759, 766, 782, 792, 798, 812, 816, 818,
       819, 822, 844, 881, 896, 901, 906, 907, 921, 929, 930, 942, 969, 971])

In [ ]:
np.where(closest_medoids == t)

In [16]:
import itertools

In [19]:
class KMedoids(object):
    def __init__(self, n_clusters, max_iter=100, metric='euclidean', random_state=None):
        self.__n_clusters = n_clusters
        self.__metric = metric
        self.__max_iter = max_iter
        self.__bags_hd_distances = None
        self.medoids = None
        self.__medoids_set = None
        self.__num_iter = 0
        self.__gamma_t = None
        self.__n_bags = 0
        self.__updated_medoids = None
        self.__converged = False
        
    def _compute_bags_distances(self, X):
        self.__bags_hd_distances = np.zeros((len(X), len(X)))
        for i in xrange(len(X)):
            self.__bags_hd_distances[i][i] = 0
            for j in xrange(i+1, len(X)):
                self.__bags_hd_distances[i][j] = haussdorf_distance(X[i], X[j])
                self.__bags_hd_distances[j][i] = self.__bags_hd_distances[i][j]
                                            
    
    def _init_medoids(self):
        self.__num_iter = 0
        self.__converged = False
        # Select k random medoids
        self.medoids = np.random.choice(self.__n_bags, size=(self.__n_clusters, ), replace=False)
        self.__medoids_set = set(self.medoids)
        self.__updated_medoids = np.copy(self.medoids)
        
    def _update_medoids(self):
        self.medoids = np.copy(self.__updated_medoids)
        self.__medoids_set = set(self.medoids)
            
    def _associate_bags_to_medoids(self):
        # Associate instances with their closest medoid.
        # N * k distances
        distances_to_medoids = self.__bags_hd_distances[:, self.medoids]
        closest_medoids = np.argmin(distances_to_medoids, axis=1) # (N, ) ndarray
        self.__gamma_t = [np.where(closest_medoids == t)[0] for t in self.medoids]
                  
    def _find_medoids(self):
        self.__updated_medoids = np.array([self._get_medoid(g) for g in self.__gamma_t])
        
    def _get_medoid(self, group):
        # G x G distances
        hd_distances = self.__bags_hd_distances[group, group]
        sum_distances = np.sum(hd_distances, axis=1)
        return groups[np.argmin(sum_distances)]
                
    def fit(self, X):
        self._compute_bags_distances(X)
        self._init_medoids(len(X))

        while (self.__num_iter == 0 or not np.array_equal(self.medoids, self.__updated_medoids))\
               and self.__num_iter < self.__max_iter:
            self._update_medoids()
            self._associate_bags_to_medoids()
            self._find_medoids()
            self.__num_iter += 1
            
        self.__converged = (self.__num_iter < self.__max_iter)
    
    def transform(self, X):
        pass
    
    def fit_transform(self, X):
        self.fit(X)
        return self.transform(X)

In [9]:
def get_medoid(bags, group):
    """Get the medoid 
    """
    if not group:
        return -1
    
    distances = np.zeros((len(group), len(group)))
    for i, g in enumerate(group):
        distances[i, :] = np.array(map(lambda j: haussdorf_distance(bags[g], bags[j]), group))
    
    sums = np.sum(distances, axis=1)
    return group[np.argmin(sums)]

In [50]:
def k_medoids(bags, k, n_iterations_max=100):
    """Perform k-medoids clustering on the bags, using Haussdorf distance.
    
    Args:
        bags (list): List of bags, each bag contains a Ni by F ndarray representing the instances.
        k (int): Number of medoids
        n_iterations_max: Maximum number of iterations if convergence is not reached.
        
    Returns:
        medoids (list): The indices of the medoids bags.
    """
    N = len(bags)
    # Select k random medoids
    medoids = np.random.choice(N, size=(k, ), replace=False)
    set_medoids = set(medoids)
    updated_medoids = np.zeros((k, ), dtype=medoids.dtype)
    
    n_iter = 0
    while not np.array_equal(medoids, updated_medoids) and n_iter < n_iterations_max:
        # Update the medoids.
        if n_iter > 0:
            medoids = np.copy(updated_medoids)
        
        # Associate instances with their closest medoid.
        gamma_t = {t: [t] for t in medoids}
        for u in xrange(N):
            if u not in set_medoids:
                du_t = map(lambda t: haussdorf_distance(bags[u], bags[t]), medoids)
                closest_t = medoids[np.argmin(du_t)]
                gamma_t[closest_t].append(u)
        
        # Update medoids
        for i, group in enumerate(gamma_t.itervalues()):
            m = get_medoid(bags, group)
            updated_medoids[i] = m
        
        n_iter += 1
                
    return medoids

In [53]:
rbags = gen_random_bags(1000, 2, 100, 128)

In [ ]:
k_medoids(rbags, 10)

In [ ]:
%timeit k_medoids(rbags, 10)